In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/Users/amitchoubey/Documents/Projects/promptEngineering/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


download the model

In [4]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [5]:
context = "The Apollo program, also known as Project Apollo, was the third United States human spaceflight program carried out by NASA, which succeeded in preparing and landing the first humans on the Moon from 1968 to 1972."
question = "Which organization was responsible for the Apollo program?"

input_text = f"""Answer the following question based on the given context.

Context: {context}

Question: {question}

Answer:"""

In [6]:
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"], max_length=100, num_beams=4, early_stopping=True)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(answer)  # Expected: NASA

NASA


Few-Shot learning 

In [7]:
examples = [
    {
        "context": "The Great Pyramid of Giza is the oldest of the Seven Wonders of the Ancient World, and the only one to remain largely intact.",
        "question": "How many Wonders of the Ancient World are there?",
        "answer": "Seven"
    },
    {
        "context": "Photosynthesis is a process used by plants to convert light energy into chemical energy that can later be released to fuel the organisms' activities.",
        "question": "What do plants convert light energy into?",
        "answer": "chemical energy"
    }
]

current_context = "The human brain contains approximately 86 billion neurons, which communicate via synaptic connections."
current_question = "How many neurons are in the human brain?"

input_text = "Answer questions based on the given contexts.\n\n"
for example in examples:
    input_text += f"Context: {example['context']}\n"
    input_text += f"Question: {example['question']}\n"
    input_text += f"Answer: {example['answer']}\n\n"

input_text += f"Context: {current_context}\n"
input_text += f"Question: {current_question}\n"
input_text += "Answer:"


In [8]:
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(answer)  # Expected: 86 billion

86 billion


CoT

In [9]:
context = "The Treaty of Versailles was signed in 1919, exactly five years after the assassination of Archduke Franz Ferdinand which occurred in 1914."
question = "In what year was Archduke Franz Ferdinand assassinated?"

input_text = f"""Read the context carefully and think step by step to answer the question.

Context: {context}

Question: {question}

Let's think step by step:
1. The Treaty of Versailles was signed in 1919
2. It was signed exactly five years after the assassination
3. Therefore, the assassination must have occurred five years before 1919
4. 1919 minus 5 years is 1914

The answer is:"""


In [10]:
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=200)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(answer)  # Expected: 1914

1. The Treaty of Versailles was signed in 1919 2. It was signed exactly five years after the assassination 3. Therefore, the assassination must have occurred five years before 1919 4. 1919 minus 5 years is 1914


Test

In [11]:
complex_context = """The Mars Rover Perseverance landed on February 18, 2021. Its mission is to search for signs of ancient life and collect rock and soil samples for possible return to Earth. Perseverance carries the Ingenuity helicopter, which became the first aircraft to achieve powered flight on another planet on April 19, 2021."""
complex_question = "How many days after Perseverance's landing did Ingenuity achieve its first flight?"

# Using chain-of-thought prompting
input_text = f"""Analyze the dates carefully and calculate the difference step by step.

Context: {complex_context}

Question: {complex_question}

Let's think step by step:
1. Perseverance landed on February 18, 2021
2. Ingenuity's first flight was on April 19, 2021
3. February has 28 days in 2021 (not a leap year)
4. Days remaining in February after landing: 28 - 18 = 10 days
5. Days in March: 31 days
6. Days in April until flight: 19 days
7. Total days = 10 (Feb) + 31 (Mar) + 19 (Apr) = 60 days

The answer is:"""

inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=300)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(answer)  # Expected: 60 days

1. Perseverance landed on February 18, 2021 2. Ingenuity's first flight was on April 19, 2021 3. February has 28 days in 2021 4. Days remaining in February after landing: 28 - 18 = 10 days 5. Days in March: 31 days 6. Days in April until flight: 19 days 7. Total days = 10 (Feb) + 31 (Mar) + 19 (Apr) = 60 days
